In [1]:
import numpy as np

import sys, os
import itertools



In [2]:
def _permutation_matrices():
    mats = []
    for perm in itertools.permutations(range(3)):
        P = np.zeros((3, 3), dtype=int)
        for i, j in enumerate(perm):
            P[i, j] = 1
        mats.append(P)
    return mats  # 6

In [3]:
pp = _permutation_matrices()
for m in pp:
    print(m)
    print()

[[1 0 0]
 [0 1 0]
 [0 0 1]]

[[1 0 0]
 [0 0 1]
 [0 1 0]]

[[0 1 0]
 [1 0 0]
 [0 0 1]]

[[0 1 0]
 [0 0 1]
 [1 0 0]]

[[0 0 1]
 [1 0 0]
 [0 1 0]]

[[0 0 1]
 [0 1 0]
 [1 0 0]]

